# Chapter 1 Challenge Solution (Python)

## Load the libraries

In [1]:
import eia_api as api
import os
import datetime
import plotly.express as px

## Question 1

Extract from the EIA dashboard the metadata of the San Diego Gas and Electric balancing authority subregion (SUBBA) under California Independent System Operator parent (PARENT). This series is under the `Hourly Demand By Subregion` sub-category.

## Solution

On the API Dashboard select the following route: 
- Electricity
- Electric Power Operation (Daily and Hourly)
- Hourly Demand by Subregion 

Once the filters loaded, go to facets and you can either use directly the `SUBBA` filter and find the San Diego Gas and Electric balancing authority subregion (out of the 83 series), or select first  California Independent System Operator on the `PARENT` filter and then it narrow down the options the four sub-regions under this parent.


If you go with the last option, you should expect to have the following API URL:

```
https://api.eia.gov/v2/electricity/rto/region-sub-ba-data/data/?frequency=hourly&data[0]=value&facets[subba][]=SDGE&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000
```

Which enables you to extract the API route path (as we saw before) - `electricity/rto/region-sub-ba-data/data`

In addition, you can extract from the header the facets argument for the GET request:


```json
{
    "frequency": "hourly",
    "data": [
        "value"
    ],
    "facets": {
        "parent": [
            "CISO"
        ],
        "subba": [
            "SDGE"
        ]
    },
    "start": null,
    "end": null,
    "sort": [
        {
            "column": "period",
            "direction": "desc"
        }
    ],
    "offset": 0,
    "length": 5000
}
```


The link to the filtered dashboard is available [here](https://www.eia.gov/opendata/browser/electricity/rto/region-sub-ba-data?frequency=hourly&data=value;&facets=subba;&subba=SDGE;&sortColumn=period;&sortDirection=desc;).

## Question 2

Set a GET request to pull observations between Jan 1st and Jan 31st 2024 with R/Python

## Solution

Using the information we pulled from the API dashboard we can set the GET request parameters:

In [3]:
api_key = os.getenv('EIA_API_KEY')

api_path = "electricity/rto/region-sub-ba-data/data/"

frequency = "hourly"

facets = {
    "parent": "CISO",
    "subba": "SDGE"
}

start = datetime.datetime(2024, 1, 1, 1)
end = datetime.datetime(2024, 1, 31, 23)

In [4]:
df1 = api.eia_get(
    api_key = api_key,
    api_path = api_path,
    frequency = frequency,
    facets = facets,
    start = start,
    end = end
)

In [5]:
df1.data

period subba                         subba-name parent  \
720 2024-01-01 00:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
719 2024-01-01 01:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
718 2024-01-01 02:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
717 2024-01-01 03:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
716 2024-01-01 04:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
..                  ...   ...                                ...    ...   
4   2024-01-30 20:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
3   2024-01-30 21:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
2   2024-01-30 22:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
1   2024-01-30 23:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
0   2024-01-31 00:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   

                                parent-name  value    value-units  
720  California Independent System Operator   2054  megawatthours  
719  California Independent System Operator   1967  megawatthours  
718  California Independent System Operator   2254  megawatthours  
717  California Independent System Operator   2541  megawatthours  
716  California Independent System Operator   2525  megawatthours  
..                                      ...    ...            ...  
4    California Independent System Operator   1109  megawatthours  
3    California Independent System Operator   1413  megawatthours  
2    California Independent System Operator   1243  megawatthours  
1    California Independent System Operator   1502  megawatthours  
0    California Independent System Operator   1706  megawatthours  

[721 rows x 7 columns]

In [6]:
px.line(df1.data, x="period", y="value")

## Question 3

Let's use the backfill function to pull data between Jan 1st 2020 and Feb 1st 2024:

In [7]:
start = datetime.datetime(2020, 1, 1, 1)
end = datetime.datetime(2024, 2, 1, 23)
offset = 2000

df2 = api.eia_backfill(
  start = start,
  end = end,
  offset = offset,
  api_path= api_path,
  api_key = api_key,
  facets = facets)

In [8]:
df2.data

period subba                         subba-name parent  \
1990 2020-01-01 00:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
1989 2020-01-01 01:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
1988 2020-01-01 02:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
1987 2020-01-01 03:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
1986 2020-01-01 04:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
...                  ...   ...                                ...    ...   
4    2024-01-31 20:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
3    2024-01-31 21:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
2    2024-01-31 22:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
1    2024-01-31 23:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   
0    2024-02-01 00:00:00  SDGE  San Diego Gas and Electric - CISO   CISO   

                                 parent-name  value    value-units  
1990  California Independent System Operator   1989  megawatthours  
1989  California Independent System Operator   2355  megawatthours  
1988  California Independent System Operator   2654  megawatthours  
1987  California Independent System Operator   2647  megawatthours  
1986  California Independent System Operator   2539  megawatthours  
...                                      ...    ...            ...  
4     California Independent System Operator   1490  megawatthours  
3     California Independent System Operator   1403  megawatthours  
2     California Independent System Operator   1313  megawatthours  
1     California Independent System Operator   1323  megawatthours  
0     California Independent System Operator   1517  megawatthours  

[35812 rows x 7 columns]

In [9]:
px.line(df2.data, x="period", y="value")